<a href="https://colab.research.google.com/github/StebMart/FRALDEA-FaceRec/blob/main/FRALDEA_Project_(COSC_4394F24).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Use the [DroneFace dataset](https://huggingface.co/docs/transformers/) from here

In [ ]:
%pip install transformers

Install the necessary packages

In [ ]:
!pip install retina-face tensorflow_addons timm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 17.4 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.4.1
    Uninstalling typeguard-4.4.1:
      Successfully uninstalled typeguard-4.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.4.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


# Uploading dataset


In [ ]:
from google.colab import drive
import zipfile
import os

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Set the path to the zip file in Google Drive
zip_path = '/content/drive/My Drive/Colab Notebooks/DnHFaces.zip'
extract_to = '/content/DnHFaces'

# Step 1: Unzip the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f"Dataset extracted to: {extract_to}")

# Verify the extracted files
print("Contents of extracted folder:")
!ls "/content/DnHFaces/open_data_set"

testing_methods = True

MessageError: Error: credential propagation was unsuccessful

# Metadata Parsing

In [ ]:
def parse_filename(filename):
    """
    Parse the filename to extract metadata such as subject ID, camera type, and image type.
    """
    parts = filename.split("_")
    metadata = {
        "subject_id": parts[0],  # Subject ID
        "camera_type": parts[1] if len(parts) > 1 else "na",
        "height_id": parts[2] if len(parts) > 2 else "na",
        "image_type": parts[3] if len(parts) > 3 else "na",
        "distance_id": parts[4].split(".")[0] if len(parts) > 4 else "na",
    }
    return metadata

if testing_methods:
    filename = "a_gp_0_ef_01.jpg"
    metadata = parse_filename(filename)
    print(metadata)

# Detect and Crop Faces Using RetinaFace


In [ ]:
from retinaface import RetinaFace
import cv2
from google.colab.patches import cv2_imshow

def detect_and_crop_face(image_path, target_size=(224, 224)):
    """
    Detect and crop faces from an image using RetinaFace.
    """
    try:
        detections = RetinaFace.detect_faces(image_path)
        if not isinstance(detections, dict) or len(detections) == 0:
            return None

        # Extract face coordinates
        face_key = list(detections.keys())[0]
        x1, y1, x2, y2 = detections[face_key]['facial_area']
        img = cv2.imread(image_path)
        if img is None or img.size == 0:
            return None

        cropped_face = img[y1:y2, x1:x2]
        cropped_face = cv2.resize(cropped_face, target_size)
        return cropped_face
    except Exception as e:
        return None

# Testing the method
image_path = "/content/DnHFaces/open_data_set/photos_all_faces/a_gp_0_ef_01.jpg"
cropped_face = detect_and_crop_face(image_path)
if cropped_face is not None:
    print("Cropped face detected successfully.")
    cv2_imshow(cropped_face)


In [ ]:
def preprocess_with_retinaface(folder_path, valid_image_types, resize_to=(224, 224)):
    images, labels, metadata_list = [], [], []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".jpg"):
                metadata = parse_filename(file)
                if metadata["image_type"] in valid_image_types:
                    img_path = os.path.join(root, file)
                    cropped_face = detect_and_crop_face(img_path, target_size=resize_to)
                    if cropped_face is not None:
                        images.append(cropped_face)
                        labels.append(metadata["subject_id"])
                        metadata_list.append(metadata)
    return images, labels, metadata_list



# Generate Proxy Portraits


In [ ]:
# def generate_proxy_portraits(images, labels, metadata, missing_subjects, valid_image_types=["ef", "porF"]):
#     """
#     Generate proxy portraits for missing subjects using images from other folders.
#     """
#     proxy_images = []
#     proxy_labels = []

#     for img, label, meta in zip(images, labels, metadata.to_dict(orient="records")):
#         if label in missing_subjects and meta["image_type"] in valid_image_types:
#             proxy_images.append(img)
#             proxy_labels.append(label)

#     return np.array(proxy_images), np.array(proxy_labels)



# Preprocess Dataset and Handle Missing Portraits

In [ ]:
def generate_proxy_portraits(images, labels, metadata, missing_subjects, valid_image_types=["ef", "porF"]):
    proxy_images, proxy_labels = [], []
    for img, label, meta in zip(images, labels, metadata):
        if label in missing_subjects and meta["image_type"] in valid_image_types:
            proxy_images.append(img)
            proxy_labels.append(label)
    return proxy_images, proxy_labels


# Preprocessing Valid Image Types

In [ ]:
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import train_test_split
# # Step 1: Preprocess dataset (already implemented)
# data_folder = "/content/DnHFaces/open_data_set"
# valid_image_types = ["ef", "por", "porF", "porL", "porR"]

# images, labels, metadata = preprocess_with_retinaface(data_folder, valid_image_types)

# # Detect missing subjects in portraits and handle them
# portraits_subjects = metadata[metadata["image_type"] == "por"]["subject_id"].unique()
# missing_subjects = set(metadata["subject_id"].unique()) - set(portraits_subjects)

# proxy_images, proxy_labels = generate_proxy_portraits(images, labels, metadata, missing_subjects)

# # Combine original and proxy portraits
# original_images = images[metadata["image_type"] == "por"]
# original_labels = labels[metadata["image_type"] == "por"]

# combined_images = np.concatenate([original_images, proxy_images], axis=0)
# combined_labels = np.concatenate([original_labels, proxy_labels], axis=0)



# Label Mapping

In [ ]:
# # Create a mapping from subject IDs to numerical labels
# unique_labels = sorted(set(combined_labels))  # Get all unique labels
# label_to_int = {label: i for i, label in enumerate(unique_labels)}  # Map each label to an integer

# # Apply the mapping to convert labels to integers
# encoded_labels = np.array([label_to_int[label] for label in combined_labels])

# Data Partitioning (5-fold Cross-Validation)

In [ ]:
from sklearn.model_selection import KFold, train_test_split
import tensorflow as tf

def k_fold_partition(images, labels, metadata, k=5):
    unique_subject_ids = list(set(labels))
    kf = KFold(n_splits=k, shuffle=True, random_state=42)

    folds = []
    for train_val_idx, test_idx in kf.split(unique_subject_ids):
        train_val_ids = [unique_subject_ids[i] for i in train_val_idx]
        test_ids = [unique_subject_ids[i] for i in test_idx]
        train_ids, val_ids = train_test_split(train_val_ids, test_size=0.2, random_state=42)

        # Partition data
        train_data, train_labels = [], []
        val_data, val_labels = [], []
        test_data, test_labels = [], []

        for img, label in zip(images, labels):
            if label in train_ids:
                train_data.append(img)
                train_labels.append(label)
            elif label in val_ids:
                val_data.append(img)
                val_labels.append(label)
            elif label in test_ids:
                test_data.append(img)
                test_labels.append(label)

        folds.append({
            "train": (train_data, train_labels),
            "val": (val_data, val_labels),
            "test": (test_data, test_labels)
        })
    return folds


In [ ]:
# # Validate all images before training
# for fold in folds:
#     train_data, train_labels = fold["train"]
#     for img in train_data:
#         assert img.shape == (224, 224, 3), f"Image shape mismatch: {img.shape}"


# Adaface Loss

In [ ]:
class AdaFaceLoss(tf.keras.losses.Loss):
    def __init__(self, scale=64, margin=0.5, name="AdaFaceLoss"):
        super(AdaFaceLoss, self).__init__(name=name)
        self.scale = scale
        self.margin = margin

    def call(self, y_true, y_pred):
        norm_logits = tf.norm(y_pred, axis=1, keepdims=True)
        quality_indicator = tf.sigmoid(norm_logits)
        adaptive_margin = self.margin * (1 - quality_indicator)
        logits_with_margin = self.scale * (tf.cos(y_pred - adaptive_margin) - adaptive_margin)
        cross_entropy_loss = tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=logits_with_margin)
        return tf.reduce_mean(cross_entropy_loss)


In [ ]:
# import numpy as np
# import torch
# from torchvision import transforms

# # Define a preprocessing pipeline for the input images
# def preprocess_images(images):
#     """
#     Preprocess images for ViT input: resize, normalize, and permute dimensions.
#     """
#     # Convert NumPy arrays to PyTorch tensors
#     images = np.array(images, dtype=np.float32)  # Ensure float32 for PyTorch compatibility
#     images = torch.tensor(images).permute(0, 3, 1, 2)  # Change shape to (batch_size, channels, height, width)

#     # Normalize images to match ImageNet pretrained weights
#     transform = transforms.Normalize(
#         mean=[0.485, 0.456, 0.406],  # ImageNet means
#         std=[0.229, 0.224, 0.225]    # ImageNet stds
#     )
#     images = transform(images / 255.0)  # Normalize to [0, 1] and apply ImageNet normalization
#     return images


# ViT Model

In [ ]:
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import EfficientNetB0

def build_vit_model(input_shape=(224, 224, 3), num_classes=11):
    base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=input_shape, pooling="avg")
    inputs = tf.keras.Input(shape=input_shape)
    x = layers.Dropout(0.5)(base_model(inputs, training=False))
    outputs = layers.Dense(num_classes, activation="softmax")(x)
    return Model(inputs, outputs)


# Ranking Evaluation

In [ ]:
# import numpy as np
# from sklearn.metrics.pairwise import cosine_similarity

# def compute_rank1_accuracy(probe_embeddings, probe_labels, gallery_embeddings, gallery_labels):
#     """
#     Compute the Rank-1 accuracy by comparing probe embeddings to gallery embeddings.
#     """
#     similarities = cosine_similarity(probe_embeddings, gallery_embeddings)
#     correct_matches = 0

#     for i, probe_label in enumerate(probe_labels):
#         # Find the index of the most similar gallery image
#         most_similar_idx = np.argmax(similarities[i])
#         predicted_label = gallery_labels[most_similar_idx]

#         # Check if the predicted label matches the probe label
#         if predicted_label == probe_label:
#             correct_matches += 1

#     rank1_accuracy = correct_matches / len(probe_labels)
#     return rank1_accuracy

# Training

In [ ]:
import numpy as np

# Assuming `train_labels` and `val_labels` contain string labels (e.g., ['h', 'k', 'l'])
all_labels = np.concatenate([train_labels, val_labels])
unique_labels = np.unique(all_labels)  # Get unique string labels

# Create a mapping from label strings to integers
label_to_int = {label: idx for idx, label in enumerate(unique_labels)}

# Map string labels to integers
train_labels_int = np.array([label_to_int[label] for label in train_labels])
val_labels_int = np.array([label_to_int[label] for label in val_labels])

# Convert integer labels to one-hot encoding
train_labels_one_hot = tf.keras.utils.to_categorical(train_labels_int, num_classes=len(unique_labels))
val_labels_one_hot = tf.keras.utils.to_categorical(val_labels_int, num_classes=len(unique_labels))

print("One-hot encoding successful!")


In [ ]:
for fold in folds:
    # Unpack train and validation data
    train_data, train_labels = fold["train"]
    val_data, val_labels = fold["val"]

    # Calculate unique labels
    all_labels = np.concatenate([train_labels, val_labels])
    unique_labels = np.unique(all_labels)

    # Convert labels to one-hot encoding
    train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=len(unique_labels))
    val_labels = tf.keras.utils.to_categorical(val_labels, num_classes=len(unique_labels))

    print("Training Fold 1...")
    # Proceed with model training


# @@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

# Evaluation

In [ ]:
def evaluate_model(model, test_data, test_labels, gallery_data, gallery_labels):
    """
    Evaluate the model using rank-1 accuracy.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    test_data, test_labels = test_data.to(device), test_labels.to(device)
    gallery_data, gallery_labels = gallery_data.to(device), gallery_labels.to(device)

    # Get embeddings for test and gallery data
    model.eval()
    with torch.no_grad():
        _, test_embeddings = model(test_data)
        _, gallery_embeddings = model(gallery_data)

    # Calculate rank-1 accuracy
    rank_1_accuracy = evaluate_ranking(test_embeddings, test_labels, gallery_embeddings, gallery_labels)
    print(f"Rank-1 Accuracy: {rank_1_accuracy * 100:.2f}%")
    return rank_1_accuracy


In [ ]:
def main_training_evaluation(folds, num_classes):
    """
    Perform training and evaluation for each fold using cross-validation.
    """
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Create a mapping from unique labels to integer indices
    unique_labels = sorted(set(label for fold in folds for label in fold["train"][1]))
    label_to_idx = {label: idx for idx, label in enumerate(unique_labels)}

    # Iterate over each fold
    for fold_idx, fold in enumerate(folds):
        print(f"\n=== Training Fold {fold_idx + 1} ===")

        # Load fold data
        train_data, train_labels = fold["train"]
        val_data, val_labels = fold["val"]
        test_data, test_labels = fold["test"]

        # Map string labels to integer indices
        train_labels = [label_to_idx[label] for label in train_labels]
        val_labels = [label_to_idx[label] for label in val_labels]
        test_labels = [label_to_idx[label] for label in test_labels]

        # Convert data and labels to PyTorch tensors
        train_data, train_labels = torch.tensor(train_data).float(), torch.tensor(train_labels).long()
        val_data, val_labels = torch.tensor(val_data).float(), torch.tensor(val_labels).long()
        test_data, test_labels = torch.tensor(test_data).float(), torch.tensor(test_labels).long()

        # Initialize model, loss function, and optimizer
        model = ViTModel(num_classes=num_classes).to(device)
        loss_function = AdaFaceLoss(embedding_size=512, num_classes=num_classes)
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

        # Train the model
        model = train_model(
            model, train_data, train_labels, val_data, val_labels,
            loss_function, optimizer, num_epochs=10
        )

        # Evaluate the model
        print(f"\n=== Evaluating Fold {fold_idx + 1} ===")
        gallery_data, gallery_labels = val_data, val_labels  # Use validation set as gallery for simplicity
        rank_1_accuracy = evaluate_model(model, test_data, test_labels, gallery_data, gallery_labels)


# Main Pipeline

In [ ]:
if __name__ == "__main__":
    folder_path = "/content/DnHFaces/open_data_set"
    valid_image_types = ["ef", "por", "porF", "porL", "porR"]

    # Preprocess the dataset
    images, labels, metadata = preprocess_with_retinaface(folder_path, valid_image_types)

    # Generate 5-fold cross-validation splits
    folds = k_fold_partition(images, labels, metadata)

    # Determine the number of unique classes
    num_classes = len(set(labels))

    # Train and evaluate the model
    main_training_evaluation(folds, num_classes)


# =============================================================================================================================

# Training Phase

In [ ]:
# Step 3: Define the Vision Transformer (ViT) model with AdaFace loss
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import EfficientNetB0

def build_vit_model(input_shape=(224, 224, 3), num_classes=11):
    """
    Build a Vision Transformer (ViT) model with an AdaFace loss.
    """
    # Base model (EfficientNet or ViT backbone)
    base_model = EfficientNetB0(include_top=False, input_shape=input_shape, pooling="avg")

    # MLP Head
    inputs = tf.keras.Input(shape=input_shape)
    features = base_model(inputs, training=False)
    outputs = layers.Dense(num_classes, activation="softmax")(features)

    return Model(inputs, outputs)

# Instantiate the model
vit_model = build_vit_model()

# Compile the model with AdaFace Loss
vit_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss=AdaFaceLoss(scale=64, margin=0.5, regularizer_weight=0.01),  # AdaFace loss
    metrics=["accuracy"]
)



# Validation Phase


In [ ]:
# Step 4: Train and validate the model for each fold
rank_1_accuracies = []

for i, fold in enumerate(folds):
    print(f"Training Fold {i + 1}...")

    train_data, train_labels = fold["train"]
    val_data, val_labels = fold["val"]

    vit_model.fit(
        train_data,
        tf.keras.utils.to_categorical(train_labels, num_classes=len(unique_labels)),
        validation_data=(val_data, tf.keras.utils.to_categorical(val_labels, num_classes=len(unique_labels))),
        epochs=10,
        batch_size=32
    )


    # Validation metrics
    val_predictions = vit_model.predict(val_data)
    val_accuracy = tf.keras.metrics.CategoricalAccuracy()
    val_accuracy.update_state(tf.keras.utils.to_categorical(val_labels), val_predictions)
    print(f"Validation Accuracy for Fold {i + 1}: {val_accuracy.result().numpy()}")


# ++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# Training Functions

In [ ]:
import torch.optim as optim

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training function
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    return running_loss / len(dataloader)

# Validation function
def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return running_loss / len(dataloader), accuracy


# Training the Model

In [ ]:
# Set number of epochs
num_epochs = 10

for epoch in range(num_epochs):
    train_loss = train(model, dataloader, criterion, optimizer, device)
    val_loss, val_accuracy = validate(model, dataloader, criterion, device)

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

    # Early stopping condition
    if val_accuracy >= 100.0:
        print(f"Reached 100% accuracy at epoch {epoch+1}. Stopping early.")
        break



# Data Preperation

In [ ]:
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class DroneFaceDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.subject_label_map = {}
        current_label = 0

        # Iterate through each subfolder (e.g., 'photos_all', 'photos_all_faces', etc.)
        for subfolder in os.listdir(root_dir):
            subfolder_path = os.path.join(root_dir, subfolder)
            if os.path.isdir(subfolder_path):
                for filename in os.listdir(subfolder_path):
                    if filename.endswith(('.jpg', '.png', '.jpeg')):
                        # Handle single-subject folders (e.g., 'photos_all_faces', 'portraits', etc.)
                        if subfolder != 'photos_all':
                            subject = filename[0].lower()  # Extract the first letter as the subject label
                            if subject not in self.subject_label_map:
                                self.subject_label_map[subject] = current_label
                                current_label += 1
                            label = self.subject_label_map[subject]
                            self.image_paths.append(os.path.join(subfolder_path, filename))
                            self.labels.append(label)
                        # Handle the 'photos_all' folder with combined subjects (e.g., 'ab')
                        else:
                            subjects_combination = filename[:2].lower()  # Extract the first two letters
                            # For simplicity, we treat combined subjects as multi-class with a placeholder label
                            label = self.subject_label_map.get(subjects_combination[0], -1)
                            self.image_paths.append(os.path.join(subfolder_path, filename))
                            self.labels.append(label)  # Adjust this for more complex handling if needed

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# Define transformations for the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize for ViT input
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load the dataset
dataset = DroneFaceDataset(root_dir='/content/DnHFaces/open_data_set', transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


# Load ViT Model


In [ ]:
import timm
import torch
import torch.nn as nn

# Load a pre-trained ViT model
model = timm.create_model('vit_base_patch16_224', pretrained=True)

# Update the classifier head for the number of classes (subjects) in the dataset
num_classes = len(set(dataset.labels))  # Number of unique subjects
model.head = nn.Linear(model.head.in_features, num_classes)

# Move the model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)


# AdaFace Loss

In [ ]:
import torch.nn.functional as F

class AdaFaceLoss(nn.Module):
    def __init__(self, s=64.0, m=0.4):
        super(AdaFaceLoss, self).__init__()
        self.s = s  # Scaling factor
        self.m = m  # Margin

    def forward(self, embeddings, labels):
        # Normalize the embeddings
        embeddings = F.normalize(embeddings)

        # Compute cosine similarity between embeddings
        cosine_similarity = F.linear(embeddings, embeddings)

        # Adjust the margin for positive pairs (same class)
        positive_margin = cosine_similarity - self.m
        logits = torch.where(labels.unsqueeze(1) == labels.unsqueeze(0), positive_margin, cosine_similarity)

        # Scale the logits
        logits = self.s * logits

        # Apply cross-entropy loss on the scaled logits
        return F.cross_entropy(logits, labels)

# Training + Validation

In [ ]:
import torch.optim as optim

# Define the loss function and optimizer
criterion = AdaFaceLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training function
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        embeddings = model(images)
        loss = criterion(embeddings, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    return running_loss / len(dataloader)

# Validation function
def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            embeddings = model(images)
            loss = criterion(embeddings, labels)
            running_loss += loss.item()

            # Calculate accuracy
            _, predicted = torch.max(embeddings, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return running_loss / len(dataloader), accuracy


# Train the Model

In [ ]:
# Set number of epochs
num_epochs = 10

for epoch in range(num_epochs):
    train_loss = train(model, dataloader, criterion, optimizer, device)
    val_loss, val_accuracy = validate(model, dataloader, criterion, device)

    print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

    # Early stopping condition
    if val_accuracy >= 100.0:
        print(f"Reached 100% accuracy at epoch {epoch+1}. Stopping early.")
        break
